In [1]:
import pandas as pd 
import os
import sys
import pandas as pd
import numpy as np
from torch.utils.data import *
from tqdm import tqdm

sys.path.append('../src/')
sys.path.append('..')

from src.models.lib.neural import GeneClassifier


In [2]:
from src.models.lib.data import *
from src.helper import *

In [3]:
# files = os.listdir('../data/processed/labels/')
# files

In [4]:
# test2 = pd.read_csv('../data/processed/labels/allen_cortex_labels.csv').set_index('cell')
# test2

In [5]:
# test2.iloc[47508, :]

In [6]:
test = GeneExpressionData(
    filename='../data/interim/allen_cortex_T.csv',
    labelname='../data/processed/labels/allen_cortex_labels.csv',
    class_label='Type',
    cast=True,
    skip=3,
)

test.__getitem__(0)

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), 7)

In [7]:
len(test.labels)

10

In [8]:
%%time

refgenes = gene_intersection()

CPU times: user 2.29 s, sys: 192 ms, total: 2.49 s
Wall time: 2.64 s


In [9]:
len(set(refgenes).intersection(test.columns))

16604

In [10]:
from torch.utils.data import DataLoader, ConcatDataset

loader = DataLoader(test, batch_size=4)
sample = next(iter(loader))
sample = sample[0].numpy()

In [11]:
def clean_sample(sample, refgenes, currgenes):
    intersection = np.intersect1d(currgenes, refgenes, return_indices=True)
    indices = intersection[1] # List of indices in currgenes that equal refgenes 
    
    axis = (1 if sample.ndim == 2 else 0)
    sample = np.sort(sample, axis=axis)
    sample = np.take(sample, indices, axis=axis)

    return torch.from_numpy(sample)

In [12]:
datafiles, labelfiles = list(INTERIM_DATA_AND_LABEL_FILES_LIST.keys()), list(INTERIM_DATA_AND_LABEL_FILES_LIST.values())

datafiles = [os.path.join('..', 'data', 'interim', f) for f in datafiles]
labelfiles = [os.path.join('..', 'data', 'processed/labels', f) for f in labelfiles]
datafiles, labelfiles

(['../data/interim/primary_bhaduri_T.csv',
  '../data/interim/allen_cortex_T.csv',
  '../data/interim/allen_m1_region_T.csv',
  '../data/interim/whole_brain_bhaduri_T.csv'],
 ['../data/processed/labels/primary_bhaduri_labels.csv',
  '../data/processed/labels/allen_cortex_labels.csv',
  '../data/processed/labels/allen_m1_region_labels.csv',
  '../data/processed/labels/whole_brain_bhaduri_labels.csv'])

In [13]:
train = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
loader = DataLoader(train, batch_size=4)
currgenes = train.columns

In [14]:
onedsample = train[0][0]
len(onedsample)

19765

In [15]:
# %%timeit
# (clean_sample(onedsample, refgenes, currgenes))

In [16]:
# twodsample = next(iter(loader))[0]
# twodsample

In [17]:
# %%timeit

# clean_sample(twodsample, refgenes, currgenes)

In [18]:
# for X, y in tqdm(loader):
#     X = clean_sample(X, refgenes, currgenes)

In [19]:
sample.ndim

2

In [20]:
temp = pd.read_csv(datafiles[0], nrows=1, header=1).columns 

In [21]:
# cols = []
# for file in datafiles:
#     # Read in columns, split by | (since some are PVALB|PVALB), and make sure all are uppercase
#     temp = pd.read_csv(file, nrows=1, header=1).columns 
#     temp = [x.split('|')[0].upper().strip() for x in temp]
    
#     print(f'Temp is {temp[0:5]}...')
#     cols.append(set(temp))

# unique = list(set.intersection(*cols))
# unique = sorted(unique)

In [22]:
# len(unique)

In [23]:
# temp = pd.read_csv(datafiles[0], nrows=1, header=1).columns 
# temp = [x.strip().upper() for x in temp]
# l = train.features

In [24]:
# l == temp

In [25]:
# len(set(unique).intersection(l))

In [26]:
# len(set(unique))

In [27]:
# len(set(unique).intersection([x.upper().strip() for x in l]))

In [28]:
train = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
loader = DataLoader(train, batch_size=4)

model = GeneClassifier(
    N_features=len(train.columns),
    N_labels=len(train.labels)
)

Model initialized. N_features = 19765, N_labels = 9. Metrics are {'accuracy': <function accuracy at 0x7f7d833a3f70>, 'precision': <function precision at 0x7f7d833c75e0>, 'recall': <function recall at 0x7f7d833c7700>} and weighted_metrics = False


In [29]:
sample = next(iter(loader))[0]
sample

tensor([[2.8467, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.8507, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.6067, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [30]:
# %%timeit

# model(sample)

Now let's time iterating over our dataloader with and without the extra data cleaning

In [31]:
# for X, y in tqdm(loader):
#     X
#     model(X)

In [32]:
train = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
loader = DataLoader(train, batch_size=4)

model = GeneClassifier(
    N_features=len(refgenes),
    N_labels=len(train.labels)
)

# for X, y in tqdm(loader):
#     X = clean_sample(X, refgenes, train.columns)
#     model(X)

Model initialized. N_features = 16604, N_labels = 9. Metrics are {'accuracy': <function accuracy at 0x7f7d833a3f70>, 'precision': <function precision at 0x7f7d833c75e0>, 'recall': <function recall at 0x7f7d833c7700>} and weighted_metrics = False


In [33]:
loader.dataset

In [34]:
train, test, insize, numlabels, weights = generate_datasets(datafiles, labelfiles, 'Type')

In [35]:
train.datasets[0].name

'../data/interim/primary_bhaduri_T.csv'

In [36]:
datafiles

['../data/interim/primary_bhaduri_T.csv',
 '../data/interim/allen_cortex_T.csv',
 '../data/interim/allen_m1_region_T.csv',
 '../data/interim/whole_brain_bhaduri_T.csv']

In [37]:
[train.datasets[i].name for i in range(len(datafiles))]

['../data/interim/primary_bhaduri_T.csv',
 '../data/interim/allen_cortex_T.csv',
 '../data/interim/allen_m1_region_T.csv',
 '../data/interim/whole_brain_bhaduri_T.csv']

In [38]:
class TestDataset(Dataset):
    def __init__(
        self, 
        filename: str, 
        labelname: str, 
        class_label: str,
        indices: Iterable[int]=None,
        skip=2,
        cast=True,
        index_col='cell',
        refgenes=None,
    ):
        self.filename = filename
        self.name = filename # alias 

        if indices is None:
            self._labeldf = pd.read_csv(labelname).set_index(index_col)
        else:
            self._labeldf = pd.read_csv(labelname).iloc[indices, :].set_index(index_col)

        self._total_data = 0
        self._class_label = class_label

        self.skip = skip
        self.cast = cast
        self.refgenes = refgenes 

    def __getitem__(self, idx):
        # The label dataframe contains both its natural integer index, as well as a "cell" index which contains the indices of the data that we 
        # haven't dropped. This is because some labels we don't want to use, i.e. the ones with "Exclude" or "Low Quality".
        # Since we are grabbing lines from a raw file, we have to keep the original indices of interest, even though the length
        # of the label dataframe is smaller than the original index
        idx = self._labeldf.iloc[idx].name
        
        # Get label
        label = self._labeldf.loc[idx, self._class_label]
        
        # get gene expression for current cell from csv file
        # We skip some lines because we're reading directly from 
        line = linecache.getline(self.filename, idx + self.skip)
        csv_data = csv.reader([line])
        data = [x for x in csv_data][0]
        
        if self.cast:
            data = torch.from_numpy(np.array([float(x) for x in data])).float()
        
        # If we need to clean the data 
        if self.refgenes is not None:
            data = clean_sample(data, refgenes, self.columns)
            
        return data, label

    def __len__(self):
        return self._labeldf.shape[0] # number of total samples 
    
    def num_labels(self):
        return self._labeldf[self._class_label].nunique()
    
    def num_features(self):
        return len(self.__getitem__(0)[0])

    def getline(self, num):
        line = linecache.getline(self.filename, num)
        csv_data = csv.reader([line])
        data = [x for x in csv_data][0]
        
        return data 

    @cached_property # Worth caching, since this is a list comprehension on up to 50k strings. Annoying. 
    def features(self):
        data = self.getline(self.skip - 1)
        data = [x.upper().strip() for x in data]
             
        return data

    @property
    def columns(self): # Just an alias...
        return self.features

    @cached_property
    def labels(self):
        return self._labeldf.loc[:, self._class_label].unique()

In [39]:
test = TestDataset(datafiles[0], labelfiles[0], 'Type', refgenes=refgenes, skip=3)

In [40]:
test[2]

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), 4)

In [41]:
test[2][0].shape

torch.Size([16604])

In [42]:
testloader = DataLoader(test, batch_size=4)

In [43]:
# for X, y in tqdm(testloader):
#     X, y

We can see that it's much faster to clean the sample on each minibatch, since numpy clearly scales well under-the-hood. Therefore, we'll have to write a manual training loop as we can no longer use pytorch lightning.

In [44]:
from pytorch_lightning import Trainer

# train = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
# loader = DataLoader(train, batch_size=4)

# model = GeneClassifier(
#     N_features=len(train.columns),
#     load
# )

In [45]:
combined, test, insize, numlabels, weights = generate_datasets(datafiles, labelfiles, 'Type')
numlabels

18

In [46]:
train = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
trainloader = DataLoader(train, batch_size=4)

net = GeneClassifier(
    N_features=len(train.columns),
    N_labels=max(train.labels)
)

Model initialized. N_features = 19765, N_labels = 17. Metrics are {'accuracy': <function accuracy at 0x7f7d833a3f70>, 'precision': <function precision at 0x7f7d833c75e0>, 'recall': <function recall at 0x7f7d833c7700>} and weighted_metrics = False


In [54]:
loaders = []
refgenes = gene_intersection()

In [55]:
for datafile, labelfile in zip(datafiles, labelfiles):
    data = GeneExpressionData(
            datafile,
            labelfile,
            'Type',
            cast=False,
    )
    
#     print(data[0][0][0:5])
    loaders.append(data)

In [ ]:
for data in loaders:
    print(data.name)
    print(data[0][0][0:5])

../data/interim/primary_bhaduri_T.csv
['FO538757.2', 'AP006222.2', 'RP11-206L10.9', 'FAM41C', 'NOC2L']
../data/interim/allen_cortex_T.csv
['0', '0', '0', '0', '0']
../data/interim/allen_m1_region_T.csv
['DDX11L1|DDX11L1', 'WASH7P|WASH7P', 'MIR6859-1|MIR6859-1', 'MIR1302-2|MIR1302-2', 'FAM138A|FAM138A']
../data/interim/whole_brain_bhaduri_T.csv


In [49]:
loaders = [DataLoader(data, batch_size=4) for data in loaders]

[[('FO538757.2', '2.84666726145933', '0.0', '0.0'),
  ('AP006222.2', '0.0', '0.0', '1.85069577786521'),
  ('RP11-206L10.9', '0.0', '0.0', '0.0'),
  ('FAM41C', '0.0', '0.0', '0.0'),
  ('NOC2L', '0.0', '0.0', '0.0'),
  ('AGRN', '0.0', '0.0', '0.0'),
  ('C1orf159', '0.0', '0.0', '0.0'),
  ('ACAP3', '0.0', '0.0', '0.0'),
  ('CPSF3L', '0.0', '0.0', '0.0'),
  ('AURKAIP1', '0.0', '0.0', '0.0'),
  ('CCNL2', '0.0', '0.0', '0.0'),
  ('MRPL20', '0.0', '2.86352156710101', '3.34219101228421'),
  ('SSU72', '0.0', '0.0', '0.0'),
  ('RP5-832C2.5', '0.0', '0.0', '0.0'),
  ('CDK11B', '0.0', '0.0', '0.0'),
  ('SLC35E2B', '0.0', '0.0', '0.0'),
  ('CDK11A', '0.0', '0.0', '0.0'),
  ('NADK', '0.0', '0.0', '0.0'),
  ('GNB1', '0.0', '0.0', '0.0'),
  ('PRKCZ', '0.0', '0.0', '0.0'),
  ('FAAP20', '0.0', '0.0', '0.0'),
  ('SKI', '0.0', '0.0', '0.0'),
  ('RER1', '0.0', '0.0', '0.0'),
  ('FAM213B', '0.0', '0.0', '0.0'),
  ('PRDM16', '0.0', '0.0', '0.0'),
  ('WRAP73', '0.0', '0.0', '0.0'),
  ('TP73-AS1', '0.0', '0.0'

In [51]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(100):  # loop over the dataset multiple times
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

KeyboardInterrupt: 